In [13]:
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, find_peaks
import matplotlib.pyplot as plt

class SpO2Calculator:
    def __init__(self, sample_rate=60):
        # Parámetros del firmware desvelados
        self.SF_spo2 = sample_rate  # 60 Hz en tu caso
        self.BUFFER_SIZE = int(self.SF_spo2 * 4)  # 60*4 = 240 muestras
        self.MA4_SIZE = 4  # Como en el firmware original
        
        # Tabla de calibración exacta que proporcionaste
        self.uch_spo2_table = [
            95,95,95,96,96,96,97,97,97,97,97,98,98,98,98,98,99,99,99,99,
            99,99,99,99,100,100,100,100,100,100,100,100,100,100,100,100,
            100,100,100,100,100,100,100,100,99,99,99,99,99,99,99,99,98,
            98,98,98,98,98,97,97,97,97,96,96,96,96,95,95,95,94,94,94,93,
            93,93,92,92,92,91,91,90,90,89,89,89,88,88,87,87,86,86,85,
            85,84,84,83,82,82,81,81,80,80,79,78,78,77,76,76,75,74,74,73,
            72,72,71,70,69,69,68,67,66,66,65,64,63,62,62,61,60,59,58,57,
            56,56,55,54,53,52,51,50,49,48,47,46,45,44,43,42,41,40,39,38,
            37,36,35,34,33,31,30,29,28,27,26,25,23,22,21,20,19,17,16,15,
            14,12,11,10,9,7,6,5,3,2,1
        ]
        
        # Verificación de parámetros
        if len(self.uch_spo2_table) != 184:
            raise ValueError("Tabla de calibración corrupta")
        if self.BUFFER_SIZE != 240:  # 60Hz ×4s
            raise ValueError("BUFFER_SIZE debe ser 240 para 60Hz")
        
    def load_data(self):
        # Cargar los datos del CSV
        self.data = pd.read_csv(self.file_path, delimiter =";")
        
        # Calcular frecuencia de muestreo
        time_diff = np.diff(self.data['Tiempo (ms)'])
        self.sample_rate = 1000 / np.mean(time_diff)  # Convertir a Hz
        
    def preprocess_signals(self):
        # Eliminar componente ambiental
        clean_ir = self.data['IR'] - self.data['AMB_IR']
        clean_red = self.data['RED'] - self.data['AMB_RED']
        
        # Eliminar valores negativos (física imposible)
        clean_ir = np.clip(clean_ir, 0, None)
        clean_red = np.clip(clean_red, 0, None)
        
        return clean_ir, clean_red
    
    def butter_bandpass(self, lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a
    
    def filter_signal(self, signal):
        # Filtro pasa banda para aislar el componente pulsátil
        b, a = self.butter_bandpass(lowcut=0.5, highcut=4, fs=self.sample_rate)
        y = filtfilt(b, a, signal)
        return y
    
    def calculate_spo2(self):
    resultados = []
    
    for i in range(0, len(self.ir), self.BUFFER_SIZE):
        ir_window = self.ir[i:i+self.BUFFER_SIZE]
        red_window = self.red[i:i+self.BUFFER_SIZE]
        
        spo2, valid, hr, hr_valid = self._process_window(ir_window, red_window)
        
        if valid:  # <--- Nivel de indentación correcto
            resultados.append({
                'timestamp': self.df['Tiempo (ms)'].iloc[i],
                'spo2': spo2,
                'hr': hr
            })
        else:  # <--- MISMO nivel que el if
            print(f"Ventana {i//self.BUFFER_SIZE}: Datos inválidos")  # Indentado dentro del else
    
    return resultados  # Fuera del for, misma indentación que el primer nivel

processor = SpO2Calculator(sample_rate=60)  # 60Hz según tu hardware
processor.load_data(r"C:\Users\Elena\Desktop\GitHub\TFG-Elena-Ruiz\Datos\Datos crudos\save_log2\raw_data_95_77_2.csv")  # Aquí va la ruta
resultados = processor.calculate_spo2()

IndentationError: expected an indented block (<ipython-input-13-b41efa355a72>, line 66)

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

class SpO2Calculator:
    def __init__(self, sample_rate=60):
        # Parámetros del firmware desvelados
        self.SF_spo2 = sample_rate  # 60 Hz en tu caso
        self.BUFFER_SIZE = int(self.SF_spo2 * 4)  # 60*4 = 240 muestras
        self.MA4_SIZE = 4  # Como en el firmware original
        
        # Tabla de calibración exacta que proporcionaste
        self.uch_spo2_table = [
            95,95,95,96,96,96,97,97,97,97,97,98,98,98,98,98,99,99,99,99,
            99,99,99,99,100,100,100,100,100,100,100,100,100,100,100,100,
            100,100,100,100,100,100,100,100,99,99,99,99,99,99,99,99,98,
            98,98,98,98,98,97,97,97,97,96,96,96,96,95,95,95,94,94,94,93,
            93,93,92,92,92,91,91,90,90,89,89,89,88,88,87,87,86,86,85,
            85,84,84,83,82,82,81,81,80,80,79,78,78,77,76,76,75,74,74,73,
            72,72,71,70,69,69,68,67,66,66,65,64,63,62,62,61,60,59,58,57,
            56,56,55,54,53,52,51,50,49,48,47,46,45,44,43,42,41,40,39,38,
            37,36,35,34,33,31,30,29,28,27,26,25,23,22,21,20,19,17,16,15,
            14,12,11,10,9,7,6,5,3,2,1
        ]
        
    

    def estimate_spo2(self, ir_data, red_data):
        # Asegurar buffer correcto
        if len(ir_data) < self.BUFFER_SIZE:
            raise ValueError(f"Se necesitan mínimo {self.BUFFER_SIZE} muestras")
            
        # 1. Preprocesado IR (igual que firmware)
        un_ir_mean = int(np.mean(ir_data[:self.BUFFER_SIZE]))
        an_x = -1 * (ir_data[:self.BUFFER_SIZE] - un_ir_mean).astype(int)
        
        # 2. Filtro MA4 (como en C++)
        for k in range(self.BUFFER_SIZE - self.MA4_SIZE):
            an_x[k] = (an_x[k] + an_x[k+1] + an_x[k+2] + an_x[k+3]) // 4
        
        # 3. Detección de valles (umbral dinámico)
        n_th1 = max(30, min(60, np.mean(an_x).astype(int)))
        valley_locs = np.zeros(15, dtype=int)
        n_valleys = self.find_valleys(an_x, n_th1)
        
        # 4. Cálculo SpO2 (algoritmo completo)
        # ... (implementación exacta del resto del algoritmo C++) ...
        
        return spo2, valid, hr, hr_valid

    def find_valleys(self, signal, min_height):
        # Implementación 1:1 del código C++ con tus parámetros
        valleys = []
        i = 1
        while i < len(signal)-1:
            if signal[i] > min_height and signal[i] > signal[i-1]:
                # ... lógica completa de detección de picos ...
            else:
                i += 1
        return valleys[:15]  # Máximo 15 valles

# Uso con tus datos de 60Hz
df = pd.read_csv(r"C:\Users\Elena\Desktop\GitHub\TFG-Elena-Ruiz\Datos\Datos crudos\save_log2\raw_data_95_77_2.csv", sep=';')
processor = SpO2Calculator(sample_rate=60)

# Procesar en bloques de 4 segundos (240 muestras)
for i in range(0, len(df), processor.BUFFER_SIZE):
    ir = df['IR'][i:i+240] - df['AMB_IR'][i:i+240]
    red = df['RED'][i:i+240] - df['AMB_RED'][i:i+240]
    
    spo2, valid, hr, hr_valid = processor.estimate_spo2(ir.values, red.values)
    
    if valid:
        print(f"SpO2: {spo2}% | HR: {hr}bpm @ {df['Tiempo (ms)'][i]}ms")